In [223]:
import pandas as pd

In [224]:
path_data = '/Users/monicaromero/PycharmProjects/afasia_cat/estudio_datos_notebook/'

df = pd.read_csv(path_data + 'df_transcrip_chunk_info.csv', encoding='utf-8')
df

,Inicio,Fin,Marca,Transcrip_name,Duración,name_chunk_audio,name_chunk_audio_path,CIP,NumId,Gènere,TipusAfàsia,LLengWAB,Edat,Grup,QA,Fluente/No Fluente
0,2.85493,6.58522,perro,02_003_CAT_Lamina,3.73029,02_003_CAT_Lamina_2.85493_6.58522.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_003,3,1,1,2,69,1,41.2,No Fluente
1,8.32512,12.09067,IL,02_003_CAT_Lamina,3.76555,02_003_CAT_Lamina_8.32512_12.09067.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_003,3,1,1,2,69,1,41.2,No Fluente
2,12.70200,16.44938,IL,02_003_CAT_Lamina,3.74738,02_003_CAT_Lamina_12.70200_16.44938.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_003,3,1,1,2,69,1,41.2,No Fluente
3,16.85885,85.00000,IL,02_003_CAT_Lamina,68.14115,02_003_CAT_Lamina_16.85885_85.00000.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_003,3,1,1,2,69,1,41.2,No Fluente
4,9.00000,10.23467,arbol,01_001_CAT_descripcio lamina,1.23467,01_001_CAT_descripcio lamina_9.00000_10.23467.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,01_001,1,1,1,1,62,1,44.3,No Fluente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,16.63210,17.65311,IL,02_002_CAT__Conversacion,1.02101,02_002_CAT__Conversacion_16.63210_17.65311.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_002,2,2,5,2,83,1,46.8,Fluente
539,26.24561,27.68594,rosa,02_002_CAT__Conversacion,1.44033,02_002_CAT__Conversacion_26.24561_27.68594.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_002,2,2,5,2,83,1,46.8,Fluente
540,33.28301,34.76044,no,02_002_CAT__Conversacion,1.47743,02_002_CAT__Conversacion_33.28301_34.76044.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_002,2,2,5,2,83,1,46.8,Fluente
541,38.57493,39.83708,eixe,02_002_CAT__Conversacion,1.26215,02_002_CAT__Conversacion_38.57493_39.83708.wav,/Users/monicaromero/PycharmProjects/afasia_cat...,02_002,2,2,5,2,83,1,46.8,Fluente


In [225]:
df.groupby('CIP')[['Fluente/No Fluente', 'LLengWAB']].first()

,Fluente/No Fluente,LLengWAB
CIP,,
01_001,No Fluente,1
01_003,No Fluente,1
01_004,No Fluente,1
01_005,No Fluente,1
01_007,Fluente,1
02_002,Fluente,2
02_003,No Fluente,2
02_007,No Fluente,2
02_008,Fluente,2


In [226]:
df['balance_group'] = df['Fluente/No Fluente'] + '_' + df['LLengWAB'].astype(str)

# Dividir el dataset en "Fluentes" y "No Fluentes"
fluentes_df = df[df['Fluente/No Fluente'] == 'Fluente']
no_fluentes_df = df[df['Fluente/No Fluente'] == 'No Fluente']

# Dividir los "Fluentes" en entrenamiento y prueba
gss_fluentes = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
train_idx_fluentes, test_idx_fluentes = next(gss_fluentes.split(fluentes_df, groups=fluentes_df['CIP']))

fluentes_train_df = fluentes_df.iloc[train_idx_fluentes]
fluentes_test_df = fluentes_df.iloc[test_idx_fluentes]

# Dividir los "No Fluentes" en entrenamiento y prueba
gss_no_fluentes = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
train_idx_no_fluentes, test_idx_no_fluentes = next(gss_no_fluentes.split(no_fluentes_df, groups=no_fluentes_df['CIP']))

no_fluentes_train_df = no_fluentes_df.iloc[train_idx_no_fluentes]
no_fluentes_test_df = no_fluentes_df.iloc[test_idx_no_fluentes]


train_df = pd.concat([fluentes_test_df, no_fluentes_test_df])  
test_df = pd.concat([fluentes_train_df, no_fluentes_train_df])  

# Verificar que los pacientes (CIP) no se repiten entre los conjuntos
assert not set(train_df['CIP']).intersection(set(test_df['CIP'])), "Los pacientes (CIP) están repetidos entre los conjuntos."

# Verificar el balance final en ambos conjuntos
def check_balance(data, columns):
    balance = {}
    for col in columns:
        balance[col] = data[col].value_counts(normalize=True)
    return balance

# Función para imprimir el balance de forma clara
def print_balance(balance, conjunto):
    print(f"\nBalance en el {conjunto}:")
    for col, vals in balance.items():
        print(f"\nBalance de {col}:")
        print(vals)

train_balance = check_balance(train_df, ['Fluente/No Fluente', 'LLengWAB'])
test_balance = check_balance(test_df, ['Fluente/No Fluente', 'LLengWAB'])

print_balance(train_balance, 'conjunto de entrenamiento ajustado')
print_balance(test_balance, 'conjunto de prueba ajustado')

# Verificar los tamaños de los conjuntos
print(f"\nTamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba: {len(test_df)}")


Balance en el conjunto de entrenamiento ajustado:

Balance de Fluente/No Fluente:
Fluente/No Fluente
No Fluente    0.798913
Fluente       0.201087
Name: proportion, dtype: float64

Balance de LLengWAB:
LLengWAB
2    0.565217
1    0.434783
Name: proportion, dtype: float64

Balance en el conjunto de prueba ajustado:

Balance de Fluente/No Fluente:
Fluente/No Fluente
No Fluente    0.788571
Fluente       0.211429
Name: proportion, dtype: float64

Balance de LLengWAB:
LLengWAB
1    0.765714
2    0.234286
Name: proportion, dtype: float64

Tamaño del conjunto de entrenamiento: 368
Tamaño del conjunto de prueba: 175


In [227]:
train_df.groupby('CIP')[['Fluente/No Fluente', 'LLengWAB']].first()

,Fluente/No Fluente,LLengWAB
CIP,,
01_001,No Fluente,1
01_003,No Fluente,1
01_007,Fluente,1
02_002,Fluente,2
02_007,No Fluente,2


In [228]:
test_df.groupby('CIP')[['Fluente/No Fluente', 'LLengWAB']].first()

,Fluente/No Fluente,LLengWAB
CIP,,
01_004,No Fluente,1
01_005,No Fluente,1
02_003,No Fluente,2
02_008,Fluente,2


In [229]:
print(len(train_df))
print(len(test_df))

368
175


In [230]:
path = '/Users/monicaromero/PycharmProjects/afasia_cat/'
train_df.to_csv(path + 'train_df.csv', index=False, encoding='utf-8')
test_df.to_csv(path + 'test_df.csv', index=False, encoding='utf-8')